In [ ]:
import cv2
import math
import glob
import ntpath
import numpy
import pandas
import random

In [ ]:
get_tn_patients()

In [ ]:
def get_tn_patients():
            
    for subset_no in range(0, 10):

        src_path = "G:/LungCancerPredict/original/luna_raw/" + "subset" + str(subset_no) + "/"
        src_files = glob.glob(src_path + "*.mhd")
        
        count=1
        
        for src_file in src_files:
            
            print("Counter:",count)
            
            get_tn_patient(src_file)
            
            count+=1

def get_tn_patient(src_file):

    patient_id = ntpath.basename(src_file).replace(".mhd", "")
    
    #读取该病人tp_nodule  
    dst_path = "G:/LungCancerPredict/extracted/luna16_extracted_images/_labels/"
    df_tp_label = pandas.read_csv(dst_path + patient_id + "_tp_nodule.csv")
    
    #根据需要处理的数据类型，读取所有数据并排序
    src_path = "G:/LungCancerPredict/extracted/luna16_extracted_images/" + patient_id + "/"
    src_files = glob.glob(src_path + "*_i.png")
    src_files.sort()
    imgs = [cv2.imread(src_file, cv2.IMREAD_GRAYSCALE) for src_file in src_files]

    #存储该病人的tn_nodule
    labels = []
    sample_count = 1000
    tries = 0

    while len(labels) < sample_count and tries < 10000:
        
        #随机选取z坐标
        tries += 1
        coord_z = int(numpy.random.normal(len(src_files) / 2, len(src_files) / 6)) 
        coord_z = max(coord_z, 0)
        coord_z = min(coord_z, len(src_files) - 1)
        img = imgs[coord_z]
        
        #返回非零值的坐标，如果全是0则进行下一次循环
        img_canny = cv2.Canny(img.copy(), 100, 200)
        non_zero_indices = numpy.nonzero(img_canny)
        if len(non_zero_indices[0]) == 0: continue
        
        #随机选取x与y坐标
        non_zero_index = random.randint(0, len(non_zero_indices[0]) - 1)
        coord_y = non_zero_indices[0][non_zero_index]
        coord_x = non_zero_indices[1][non_zero_index]
        
        ok = True
        
        #将该edge上的点与其他所有tp_nodule进行对比
        for index, row in df_tp_label.iterrows():
            
            pos_coord_x = row["coord_x"] * img.shape[1]
            pos_coord_y = row["coord_y"] * img.shape[0]
            pos_coord_z = row["coord_z"] * len(src_files)
            diameter = row["diameter"] * img.shape[1]
            
            dist = math.sqrt(math.pow(pos_coord_x - coord_x, 2) + math.pow(pos_coord_y - coord_y, 2) + math.pow(pos_coord_z - coord_z, 2))
            
            if dist < (diameter + 48): ok = False; break
    
        if not ok: continue
    
        perc_x = round(coord_x / img.shape[1], 4)
        perc_y = round(coord_y / img.shape[0], 4)
        perc_z = round(coord_z / len(src_files), 4)
        diameter_percent = round(6 / img.shape[1], 4)
        
        labels.append([len(labels), perc_x, perc_y, perc_z, diameter_percent, 0])
    
    if tries > 9999:
        
        print("****** WARING!! TOO MANY TRIES ************************************")
    
    df_labels = pandas.DataFrame(labels, columns=["anno_index", "coord_x", "coord_y", "coord_z", "diameter", "malscore"])
    df_labels.to_csv(dst_path + patient_id + "_tn_nodule.csv", index=False)